In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

from jamie import JAMIE
from jamie.evaluation import *
from jamie.utilities import *
import matplotlib.pyplot as plt
from mmd_wrapper import mmd_combine
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import preprocessing

2022-10-27 14:35:40.627614: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-27 14:35:40.708886: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-27 14:35:40.712242: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-27 14:35:40.712254: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
np.random.seed(42)

In [6]:
dataset_name = 'scMNC-Visual'
dataset_color = 'magenta'
modality_names = ['Gene Expression', 'Electrophysiology']
ephys_dir = '../data/scMNC/mouse_visual_cortex/raw_data/'

# Gene Expression
data1 = pd.read_csv(ephys_dir + 'patch-seq-counts.csv', sep='\t')
data1.index = data1['gene']
data1 = data1.iloc[:, 1:].T
fnames1 = data1.columns
snames1 = np.array(['_'.join(d.split('_')[:-1]) for d in data1.index])
data1 = data1.to_numpy()

# Ephys
data2 = np.load(ephys_dir + 'exp.npy', allow_pickle=True)
for d in data2:
    darr = d['name'].split(' ')
    if darr[3] != 'slice':
        darr.insert(4, darr[3][6:])
        darr[3] = darr[3][:6]
    if darr[5] != 'sample':
        darr.insert(6, darr[5][7:])
        darr[5] = darr[5][:7]
    darr = np.array(darr)
    name = ''.join(darr[[0, 2, 1]])
    name += '_' + '_'.join(darr[[5, 6]])  
    name += '_L4'
    d['name'] = name
snames2 = np.array([d['name'] for d in data2])

# Corr
corr = np.zeros((len(snames1), len(snames2)))
for i, n in enumerate(snames1):
    corr[i, np.argwhere(snames2 == n)] = 1

# Meta
meta = pd.read_excel(ephys_dir + 'layer4-cells.xlsx')
# tray = row = None
for i in range(meta.shape[0]):
    # print(meta.iloc[i])
    # tray = meta.iloc[i, 0] if not pd.isna(meta.iloc[i, 0]) else tray
    # row = meta.iloc[i, 1] if not pd.isna(meta.iloc[i, 1]) else row
    if not pd.isna(meta.iloc[i, 2]):
        name = ''.join(np.array(meta.iloc[i, 2].split(' '))[[2, 0, 1]])
        name += '_' + meta.iloc[i, 3].replace(' ', '_')
        name += '_L4'
        meta.iloc[i, 3] = name
meta = meta.iloc[[not pd.isna(meta.iloc[i, 3]) for i in range(meta.shape[0])], 3:]
meta.index = meta.iloc[:, 0]
meta = meta.iloc[:, 1:]
type1 = type2 = np.array(meta.iloc[:, 0])

# Labels
labels = [type1, type2]
features = [np.array(fnames1), None]
feature_dict = {}

# Preprocessing
data1 = preprocessing.scale(data1, axis=0)
# data2 = preprocessing.scale(data2, axis=0)
data1[np.isnan(data1)] = 0  # Replace NaN with average
# data2[np.isnan(data2)] = 0
dataset = [data1, data2]

# Train-Test Imputation
train_size = int(.8 * len(data1))
train_idx = np.random.choice(range(len(data1)), train_size, replace=False)
test_idx = np.array(list(set(range(len(data1))) - set(train_idx)))

In [7]:
reduced_dim = 32

In [8]:
data1

array([[-0.469,  0.304, -0.27 , ...,  0.   ,  0.   ,  0.   ],
       [-0.052, -0.686, -0.27 , ...,  0.   ,  0.   ,  0.   ],
       [-0.469, -0.722, -0.27 , ...,  0.   ,  0.   ,  0.   ],
       ...,
       [-0.026,  0.494, -0.27 , ...,  0.   ,  0.   ,  0.   ],
       [-0.469, -0.29 ,  1.756, ...,  0.   ,  0.   ,  0.   ],
       [-0.449, -0.892, -0.27 , ...,  0.   ,  0.   ,  0.   ]])

In [9]:
data2

array([{'name': '20180703_sample_11_L4', 'stim': -200.0, 'resp': array([-62.028, -62.293, -62.599, ..., -98.758, -98.64 , -98.477])},
       {'name': '20180703_sample_11_L4', 'stim': -180.0, 'resp': array([-60.793, -61.034, -61.313, ..., -95.304, -95.196, -95.047])},
       {'name': '20180703_sample_11_L4', 'stim': -160.0, 'resp': array([-60.342, -60.56 , -60.808, ..., -92.481, -92.382, -92.249])},
       ...,
       {'name': '20180627_sample_14_L4', 'stim': 1340.0, 'resp': array([-64.809, -63.729, -62.518, ..., -27.459, -28.052, -28.779])},
       {'name': '20180627_sample_14_L4', 'stim': 1360.0, 'resp': array([-63.871, -62.722, -61.458, ..., -27.342, -27.9  , -28.589])},
       {'name': '20180627_sample_14_L4', 'stim': 1380.0, 'resp': array([-64.243, -63.087, -61.814, ..., -28.388, -28.917, -29.577])}],
      dtype=object)

In [12]:
corr

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])